---

_You are currently looking at **version 1.0** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [143]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind
import re

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
import pandas as pd
import numpy as np
from scipy import stats

In [3]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [4]:
#!cat university_towns.txt

In [5]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan","Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State","RegionName"]  )'''
    '''
    f = open('university_towns.txt', 'r')
    universities_text = pd.Series(f.readlines())
    universities_text.replace(['\(.*\)'],[''], regex=True, inplace=True)
    universities_text.replace(['\\n'],[''], regex=True, inplace=True)
    universities_text.replace(['\[\d\]'],[''], regex=True, inplace=True)
    state_indices = universities_text[universities_text.str.contains('\[edit\]')].index.tolist()
    universities_text.replace(['\[edit\]'],[''], regex=True, inplace=True)
    universities_text = universities_text.str.strip()
    UniversityTowns = pd.DataFrame(universities_text, columns=['RegionName'])
    UniversityTowns['State'] = ''
    state_indices.append(len(UniversityTowns))
    for i in range(0,len(state_indices)-1):
        UniversityTowns.loc[state_indices[i]:(state_indices[i+1]),'State'] = UniversityTowns['RegionName'].iloc[state_indices[i]]
    UniversityTowns.drop(UniversityTowns.index[state_indices[0:-1]], inplace=True)
    UniversityTowns = UniversityTowns[['State', 'RegionName']]
    UniversityTowns = UniversityTowns.reset_index(drop=True)
    '''
    df = pd.read_csv('university_towns.txt', sep="\n", header = None,names=['name'])
    university = df[df['name'].str.contains('[edit]',regex=False)].index.tolist()
    university.append(len(df))
    for i in range(1,len(university)):
        df.loc[(df.index >= university[i-1])&(df.index < university[i]),'State'] = df['name'].iloc[university[i-1]]
    df.drop(df.index[university[:-1]],inplace=True)
    df['State'] = df['State'].str.replace(r"\[.*","")
    df['RegionName'] = df['name'].str.replace(r" \(.*",'')
    df['RegionName'] = df['RegionName'].str.replace(r"\[.*","")
    del df['name']
    df = df.reset_index(drop=True)
    return df


In [43]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    gdp = pd.read_excel('gdplev.xls', usecols=range(4,7), skiprows=7)
    gdp.columns = ['Quarter', 'GDP', 'GDP Chained']
    gdp = gdp.iloc[gdp[gdp['Quarter'].str.startswith('2000')].index.tolist()[0]:].reset_index(drop=True)
    gdp['diff'] = gdp['GDP Chained'].diff()
    gdp['recession_start'] = [(gdp['diff'].iloc[i]<0) & (gdp['diff'].iloc[i-1]<0) for i,x in enumerate(gdp['diff'])]
    return gdp.iloc[gdp[gdp['recession_start'] == True].index.tolist()[0] -1]['Quarter']
    

'2008q3'

In [45]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    start = get_recession_start()
    gdp = pd.read_excel('gdplev.xls', usecols=range(4,7), skiprows=7)
    gdp.columns = ['Quarter', 'GDP', 'GDP Chained']
    gdp = gdp.iloc[gdp[gdp['Quarter'].str.startswith(start)].index.tolist()[0]:].reset_index(drop=True)
    gdp['diff'] = gdp['GDP Chained'].diff()
    gdp['recession_end'] = [(gdp['diff'].iloc[i]>0) & (gdp['diff'].iloc[i-1]>0) for i,x in enumerate(gdp['diff'])]
    return gdp.iloc[gdp[gdp['recession_end'] == True].index.tolist()[0]]['Quarter']


'2009q3'

In [180]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    start = get_recession_start()
    end = get_recession_end()
    gdp = pd.read_excel('gdplev.xls', usecols=range(4,7), skiprows=7)
    gdp.columns = ['Quarter', 'GDP', 'GDP Chained']
    gdp = gdp.iloc[gdp[gdp['Quarter'].str.startswith(start)].index.tolist()[0]:gdp[gdp['Quarter'].str.startswith(end)].index.tolist()[0]+1].reset_index(drop=True)
    return gdp[gdp['GDP Chained'] == gdp['GDP Chained'].min()]['Quarter'].tolist()[0]

In [175]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    df = pd.read_csv('City_Zhvi_AllHomes.csv')
    df.columns.tolist().index('2001-01')
    df = df.loc[:, ['State', 'RegionName','RegionID','Metro','CountyName','SizeRank'] + df.columns[63:].tolist()]
    for i in range(6,193, 3):
        quarter = re.search('\d*', df.columns[i]).group(0) + 'q'+str((int(re.search('-\d*', df.columns[i]).group(0)[1:])//3)+1)
        df[quarter] = df[df.columns[[i,i+1,i+2]]].apply(np.mean, axis=1)
    df.drop(df.columns[[range(2,190)]],axis=1, inplace=True)
    df.replace({'State' : states}, inplace=True)
    df.set_index(['State','RegionName'], inplace=True)
    return df



In [178]:
df = convert_housing_data_to_quarters()

In [191]:
lambda i for i,x in enumerate(df.columns == get_recession_start()) if x
#print(get_recession_bottom())

SyntaxError: invalid syntax (<ipython-input-191-e46cf535598f>, line 1)

In [ ]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    return "ANSWER"